In [6]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import datetime 
# Define the column names as a list
columns = ['MMYYYY','Bank_name', 'CASA_int', 'Off_1M', 'Off_3M', 'Off_6M', 'Off_1Y', 'Off_2Y', 'Onl_1M', 'Onl_3M', 'Onl_6M','Onl_1Y','Onl_2Y']

Update_inst = pd.DataFrame(columns=columns)
Update_inst

,MMYYYY,Bank_name,CASA_int,Off_1M,Off_3M,Off_6M,Off_1Y,Off_2Y,Onl_1M,Onl_3M,Onl_6M,Onl_1Y,Onl_2Y


In [8]:
url_ACB = 'https://www.acb.com.vn/lai-suat-tien-gui' # Done
url_AGR = 'https://www.agribank.com.vn/vn/lai-suat'     #Done
url_CTG = 'https://www.vietinbank.vn/web/home/vn/lai-suat' # Done


Note: Dữ liệu của ACB phía dưới là lãi suất cho tài khoản tiền gửi < 200 tỷ

In [9]:


# Function to convert a string containing ',' to a float and replace ',' with '.'
def convert_to_float(value):
    return float(value.replace(',', '.'))

# Send an HTTP GET request to the URL and retrieve the content
acb = requests.get(url_ACB)

# Set the common values for 'MMYYYY' and 'Bank_name'
Update_inst.loc[0, 'MMYYYY'] = datetime.datetime.now().strftime("%d/%m/%Y")
Update_inst.loc[0, 'Bank_name'] = 'ACB'

# Check if the request was successful (status code 200)
if acb.status_code == 200:
    html_content = acb.text
    
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Find the CASA_int value and convert it to float
    casa = soup.find_all('span', class_='font6')[0].text
    Update_inst.loc[0, 'CASA_int'] = convert_to_float(casa)

    # Find and convert offline and online values to float using list comprehension
    offline_values = [convert_to_float(soup.find_all("tbody")[1].find_all("tr")[x].find_all("td", class_="xl68")[0].text.strip()) for x in [5, 7, 10, 12,16]]
    online_values = [convert_to_float(soup.find_all("tbody")[3].find_all("tr")[2].find_all("td")[y].text.strip()) for y in [2, 4, 5, 6, 6]]

    # Update the DataFrame with the converted values
    Update_inst.loc[0, ["Off_1M", "Off_3M", "Off_6M", "Off_1Y", "Off_2Y"]] = offline_values
    Update_inst.loc[0, ["Onl_1M", "Onl_3M", "Onl_6M", "Onl_1Y", "Onl_2Y"]] = online_values


In [23]:
Update_inst

,MMYYYY,Bank_name,CASA_int,Off_1M,Off_3M,Off_6M,Off_1Y,Off_2Y,Onl_1M,Onl_3M,Onl_6M,Onl_1Y,Onl_2Y
0,30/09/2023,ACB,0.05,3.3,3.5,4.8,5.3,5.4,3.5,3.7,5.0,5.1,5.1


In [24]:
# Make a GET request to the AGR data URL
agr = requests.get(url_AGR)

# Check if the request was successful (status code 200)
if agr.status_code == 200:
    html_content = agr.text
    
    # Parse the HTML content using BeautifulSoup    
    soup = BeautifulSoup(html_content, 'html.parser')

    offline_values = []
    for x in [0, 1, 3, 6, 12, 16]:
        offline = soup.find_all("tbody")[0].find_all("tr")[x].find_all("td")[1].text.strip()
        offline_values.append(offline)

    new_row = {
        'MMYYYY': datetime.datetime.now().strftime("%d/%m/%Y"),
        'Bank_name': 'AGR',
        'CASA_int': offline_values[0],
        'Off_1M': offline_values[1],
        'Off_3M': offline_values[2],
        'Off_6M': offline_values[3],
        'Off_1Y': offline_values[4],
        'Off_2Y': offline_values[5],
    }
    
    for column in ['CASA_int', 'Off_1M', 'Off_3M', 'Off_6M', 'Off_1Y', 'Off_2Y']:
        new_row[column] = float(new_row[column].replace('%', ''))
        new_row_df = pd.DataFrame([new_row])
    
    # Append the new row to the DataFrame
    Update_inst = pd.concat([Update_inst, new_row_df], ignore_index=True)

# Print the updated DataFrame
Update_inst


,MMYYYY,Bank_name,CASA_int,Off_1M,Off_3M,Off_6M,Off_1Y,Off_2Y,Onl_1M,Onl_3M,Onl_6M,Onl_1Y,Onl_2Y
0,30/09/2023,ACB,0.05,3.3,3.5,4.8,5.3,5.4,3.5,3.7,5.0,5.1,5.1
1,30/09/2023,AGR,0.2,3.0,3.5,4.5,5.5,5.5,NaN,NaN,NaN,NaN,NaN


In [25]:
ctg = requests.get(url_CTG)

# Check if the request was successful (status code 200)
if ctg.status_code == 200:
    html_content = ctg.text
    
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

    values = []
    for x in [3,4,7,10,16,19]:
        offline = soup.find_all("tbody")[1].find_all("tr")[x].find_all("td")[1].text.strip()
        offline_values.append(offline)

    new_row = {
        'MMYYYY': datetime.datetime.now().strftime("%d/%m/%Y"),
        'Bank_name': 'CTG',
        'CASA_int': offline_values[0],
        'Off_1M': offline_values[1],
        'Off_3M': offline_values[2],
        'Off_6M': offline_values[3],
        'Off_1Y': offline_values[4],
        'Off_2Y': offline_values[5],
    }
    for column in ['CASA_int', 'Off_1M', 'Off_3M', 'Off_6M', 'Off_1Y', 'Off_2Y']:
        new_row[column] = float(new_row[column].replace('%', ''))
        new_row_ctg = pd.DataFrame([new_row])
    # Append the new row to the DataFrame
    Update_inst = pd.concat([Update_inst, new_row_ctg], ignore_index=True)

# Print the updated DataFrame
Update_inst

,MMYYYY,Bank_name,CASA_int,Off_1M,Off_3M,Off_6M,Off_1Y,Off_2Y,Onl_1M,Onl_3M,Onl_6M,Onl_1Y,Onl_2Y
0,30/09/2023,ACB,0.05,3.3,3.5,4.8,5.3,5.4,3.5,3.7,5.0,5.1,5.1
1,30/09/2023,AGR,0.2,3.0,3.5,4.5,5.5,5.5,NaN,NaN,NaN,NaN,NaN
2,30/09/2023,CTG,0.2,3.0,3.5,4.5,5.5,5.5,NaN,NaN,NaN,NaN,NaN
